In [8]:
import pandas as pd
import numpy as np

In [9]:
eth = pd.read_csv('eth.csv')

In [10]:
eth['movingAverage'] = eth.adjClose.rolling(20).mean()
eth['standardDeviation'] = eth.adjClose.rolling(20).std()

In [11]:
eth['BollingerBandsUpper'] = eth['movingAverage'] + (eth['standardDeviation'] * 2)
eth['BollingerBandsLower'] = eth['movingAverage'] - (eth['standardDeviation'] * 2)

### RSI Calculation Formula
1. RSI = 100 – 100 / ( 1 + RS )
1. RS = Relative Strength = AvgU / AvgD.
1. AvgU = average of all up moves in the last N price bars.
1. AvgD = average of all down moves in the last N price bars.
1. N = the period of RSI.

In [12]:
def rma(x, n, y0):
    a = (n-1) / n
    ak = a**np.arange(len(x)-1, -1, -1)
    return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]

In [14]:
N = 14

eth['change'] = eth['adjClose'].diff(1)

eth['gain'] = eth.change.mask(eth.change < 0, 0.0)
eth['loss'] = -eth.change.mask(eth.change > 0, -0.0)

eth['AvgU'] = rma(eth.gain[N+1:].to_numpy(), N, np.nansum(eth.gain.to_numpy()[:N+1])/N)
eth['AvgD'] = rma(eth.loss[N+1:].to_numpy(), N, np.nansum(eth.loss.to_numpy()[:N+1])/N)

eth['rs'] = abs(eth['AvgU']/eth['AvgD'])
eth['rsi'] = 100 - 100 / (1 + eth['rs'])

In [16]:
eth.to_csv('eth_1.csv', index=False)


### The Formula for the Williams %R Is:
Wiliams %R = (Highest High - Close) / (Highest High - Lowest Low)

Highest High=Highest price in the lookback period, typically 14 days.

Close=Most recent closing price.

Lowest Low=Lowest price in the lookback period, typically 14 days.